<a href="https://colab.research.google.com/github/HRI328/AXA-CASE-1/blob/main/RAG_from_multiple_data_sources.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%%bash
pip install transformers torch faiss-cpu sentence-transformers datasets accelerate
pip install pandas sqlalchemy openpyxl python-docx pypdf2 beautifulsoup4
pip install langchain chromadb

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.4/31.4 MB 24.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 253.0/253.0 kB 10.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 8.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 4.9 MB/s eta 0:00:00
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.8/20.8 MB 27.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.2/278.2 kB 14.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 68.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.3/103.3 kB 8.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-adk 1.17.0 requires opentelemetry-api<=1.37.0,>=1.37.0, but you have opentelemetry-api 1.38.0 which is incompatible.
google-adk 1.17.0 requires opentelemetry-sdk<=1.37.0,>=1.37.0, but you have opentelemetry-sdk 1.38.0 which is incompatible.
opentelemetry-exporter-otlp-proto-http 1.37.0 requires opentelemetry-exporter-otlp-proto-common==1.37.0, but you have opentelemetry-exporter-otlp-proto-common 1.38.0 which is incompatible.
opentelemetry-exporter-otlp-proto-http 1.37.0 requires opentelemetry-proto==1.37.0, but you have opentelemetry-proto 1.38.0 which is incompatible.
opentelemetry-exporter-otlp-proto-http 1.37.0 requires opentelemetry-sdk~=1.37.0, but you have opentelemetry-sdk 1.38.0 which is incompatible.


In [2]:
import torch
import pandas as pd
import numpy as np
from transformers import (
    AutoTokenizer,
    AutoModelForSeq2SeqLM,
    pipeline,
    AutoModel
)
from sentence_transformers import SentenceTransformer
import faiss
from typing import List, Dict, Any, Union
import json
import re
from sqlalchemy import create_engine, text
import os
from pathlib import Path

class DataProcessor:
    """Process both structured and unstructured data"""

    def __init__(self, chunk_size: int = 512, overlap: int = 50):
        self.chunk_size = chunk_size
        self.overlap = overlap

    def process_unstructured_text(self, text: str) -> List[Dict[str, Any]]:
        """Process unstructured text documents"""
        chunks = []
        sentences = text.split('. ')

        current_chunk = ""
        for sentence in sentences:
            if len(current_chunk.split()) + len(sentence.split()) <= self.chunk_size:
                current_chunk += sentence + ". "
            else:
                if current_chunk:
                    chunks.append({
                        'content': current_chunk.strip(),
                        'type': 'unstructured',
                        'metadata': {'chunk_size': len(current_chunk.split())}
                    })
                current_chunk = sentence + ". "

        if current_chunk:
            chunks.append({
                'content': current_chunk.strip(),
                'type': 'unstructured',
                'metadata': {'chunk_size': len(current_chunk.split())}
            })

        return chunks

    def process_structured_data(self, data: Union[pd.DataFrame, List[Dict]]) -> List[Dict[str, Any]]:
        """Process structured data (DataFrame or list of dicts)"""
        chunks = []

        if isinstance(data, pd.DataFrame):
            # Convert DataFrame to list of records
            records = data.to_dict('records')
        else:
            records = data

        for record in records:
            # Convert each record to a readable text format
            record_text = "Structured data record: "
            for key, value in record.items():
                record_text += f"{key}: {value}, "

            chunks.append({
                'content': record_text.rstrip(', '),
                'type': 'structured',
                'metadata': {
                    'record_type': 'structured_record',
                    'fields': list(record.keys()) if record else []
                }
            })

        return chunks

    def process_sql_data(self, connection_string: str, query: str) -> List[Dict[str, Any]]:
        """Process data from SQL database"""
        try:
            engine = create_engine(connection_string)
            with engine.connect() as conn:
                result = conn.execute(text(query))
                rows = result.fetchall()
                columns = result.keys()

            chunks = []
            for row in rows:
                row_dict = dict(zip(columns, row))
                row_text = "Database record: "
                for key, value in row_dict.items():
                    row_text += f"{key}: {value}, "

                chunks.append({
                    'content': row_text.rstrip(', '),
                    'type': 'structured',
                    'metadata': {
                        'source': 'sql_database',
                        'table_columns': list(columns)
                    }
                })

            return chunks
        except Exception as e:
            print(f"Error processing SQL data: {e}")
            return []

    def process_csv(self, file_path: str) -> List[Dict[str, Any]]:
        """Process CSV file"""
        df = pd.read_csv(file_path)
        return self.process_structured_data(df)

    def process_json(self, file_path: str) -> List[Dict[str, Any]]:
        """Process JSON file"""
        with open(file_path, 'r') as f:
            data = json.load(f)

        if isinstance(data, list):
            return self.process_structured_data(data)
        else:
            # Convert single JSON object to list
            return self.process_structured_data([data])

    def process_excel(self, file_path: str, sheet_name: str = None) -> List[Dict[str, Any]]:
        """Process Excel file"""
        if sheet_name:
            df = pd.read_excel(file_path, sheet_name=sheet_name)
        else:
            df = pd.read_excel(file_path)
        return self.process_structured_data(df)

class MultiModalVectorDatabase:
    """Vector database that handles both structured and unstructured data"""

    def __init__(self, embedding_model_name: str = "all-MiniLM-L6-v2"):
        self.embedding_model = SentenceTransformer(embedding_model_name)
        self.index = None
        self.documents = []
        self.metadata = []

    def add_documents(self, documents: List[Dict[str, Any]]):
        """Add documents with metadata to the vector database"""
        self.documents.extend(documents)

        # Extract content for embedding
        contents = [doc['content'] for doc in documents]
        metadata_list = [doc.get('metadata', {}) for doc in documents]
        doc_types = [doc.get('type', 'unknown') for doc in documents]

        self.metadata.extend([
            {**meta, 'type': doc_type}
            for meta, doc_type in zip(metadata_list, doc_types)
        ])

        # Generate embeddings
        embeddings = self.embedding_model.encode(contents, show_progress_bar=True)

        # Create or update FAISS index
        if self.index is None:
            dimension = embeddings.shape[1]
            self.index = faiss.IndexFlatIP(dimension)

        # Normalize embeddings for cosine similarity
        faiss.normalize_L2(embeddings)
        self.index.add(embeddings.astype('float32'))

    def search(self, query: str, k: int = 5, doc_type: str = None) -> List[Dict[str, Any]]:
        """Search for similar documents with optional type filtering"""
        if self.index is None:
            raise ValueError("Index not built. Add documents first.")

        # Encode query
        query_embedding = self.embedding_model.encode([query])
        faiss.normalize_L2(query_embedding)

        # Search
        scores, indices = self.index.search(query_embedding.astype('float32'), k * 2)  # Get more for filtering

        # Filter and return results
        results = []
        for score, idx in zip(scores[0], indices[0]):
            if idx < len(self.documents):
                doc = self.documents[idx]
                meta = self.metadata[idx]

                # Apply type filter if specified
                if doc_type and meta.get('type') != doc_type:
                    continue

                results.append({
                    'content': doc['content'],
                    'score': float(score),
                    'index': int(idx),
                    'type': meta.get('type', 'unknown'),
                    'metadata': meta
                })

                if len(results) >= k:
                    break

        return results

    def hybrid_search(self, query: str, k: int = 5) -> Dict[str, List[Dict[str, Any]]]:
        """Perform hybrid search across both data types"""
        # Get results from both types
        structured_results = self.search(query, k, doc_type='structured')
        unstructured_results = self.search(query, k, doc_type='unstructured')

        return {
            'structured': structured_results,
            'unstructured': unstructured_results
        }

class AdvancedRAGSystem:
    def __init__(self,
                 generator_model: str = "google/flan-t5-large",
                 embedding_model: str = "all-mpnet-base-v2"):

        # Initialize components
        self.data_processor = DataProcessor()
        self.vector_db = MultiModalVectorDatabase(embedding_model)

        # Initialize generator
        self.tokenizer = AutoTokenizer.from_pretrained(generator_model)
        self.model = AutoModelForSeq2SeqLM.from_pretrained(generator_model)

        self.generator = pipeline(
            "text2text-generation",
            model=self.model,
            tokenizer=self.tokenizer,
            device=0 if torch.cuda.is_available() else -1,
            max_length=512
        )

    def add_structured_data(self, data: Union[pd.DataFrame, List[Dict], str]):
        """Add structured data to the system"""
        if isinstance(data, str):
            # Assume it's a file path
            file_ext = Path(data).suffix.lower()
            if file_ext == '.csv':
                documents = self.data_processor.process_csv(data)
            elif file_ext == '.json':
                documents = self.data_processor.process_json(data)
            elif file_ext in ['.xlsx', '.xls']:
                documents = self.data_processor.process_excel(data)
            else:
                raise ValueError(f"Unsupported file format: {file_ext}")
        else:
            documents = self.data_processor.process_structured_data(data)

        self.vector_db.add_documents(documents)
        return f"Added {len(documents)} structured documents"

    def add_unstructured_data(self, text: str):
        """Add unstructured text to the system"""
        documents = self.data_processor.process_unstructured_text(text)
        self.vector_db.add_documents(documents)
        return f"Added {len(documents)} unstructured documents"

    def add_sql_data(self, connection_string: str, query: str):
        """Add data from SQL database"""
        documents = self.data_processor.process_sql_data(connection_string, query)
        self.vector_db.add_documents(documents)
        return f"Added {len(documents)} database records"

    def retrieve_context(self, query: str, k: int = 3) -> Dict[str, Any]:
        """Retrieve context from both data types"""
        hybrid_results = self.vector_db.hybrid_search(query, k)

        # Combine and rank all results
        all_results = hybrid_results['structured'] + hybrid_results['unstructured']
        all_results.sort(key=lambda x: x['score'], reverse=True)

        return {
            'structured': hybrid_results['structured'],
            'unstructured': hybrid_results['unstructured'],
            'combined': all_results[:k*2]  # Return top combined results
        }

    def generate_structured_prompt(self, query: str, structured_context: List[Dict]) -> str:
        """Create prompt specifically for structured data queries"""
        if not structured_context:
            return ""

        context_text = "Structured Data Context:\n"
        for i, result in enumerate(structured_context):
            context_text += f"{i+1}. {result['content']}\n"

        return context_text

    def generate_unstructured_prompt(self, query: str, unstructured_context: List[Dict]) -> str:
        """Create prompt specifically for unstructured data queries"""
        if not unstructured_context:
            return ""

        context_text = "Unstructured Data Context:\n"
        for i, result in enumerate(unstructured_context):
            context_text += f"{i+1}. {result['content']}\n"

        return context_text

    def generate_hybrid_prompt(self, query: str, context: Dict[str, Any]) -> str:
        """Create comprehensive hybrid prompt"""
        structured_prompt = self.generate_structured_prompt(query, context['structured'])
        unstructured_prompt = self.generate_unstructured_prompt(query, context['unstructured'])

        prompt = f"""You are an AI assistant with access to both structured and unstructured data.

{structured_prompt}
{unstructured_prompt}

Question: {query}

Please provide a comprehensive answer based on the available data. If the structured data contains relevant information, use it for precise answers. Use unstructured data for contextual understanding.

Answer:"""
        return prompt

    def analyze_query_type(self, query: str) -> Dict[str, bool]:
        """Analyze query to determine which data types are most relevant"""
        # Simple heuristic-based analysis
        structured_keywords = [
            'how many', 'count', 'sum', 'average', 'maximum', 'minimum',
            'list of', 'table', 'database', 'record', 'field'
        ]

        unstructured_keywords = [
            'explain', 'describe', 'what is', 'how does', 'why',
            'concept', 'theory', 'definition'
        ]

        query_lower = query.lower()

        return {
            'needs_structured': any(keyword in query_lower for keyword in structured_keywords),
            'needs_unstructured': any(keyword in query_lower for keyword in unstructured_keywords)
        }

    def generate(self, query: str, k: int = 3) -> Dict[str, Any]:
        """Generate answer using hybrid RAG approach"""
        # Analyze query type
        query_analysis = self.analyze_query_type(query)

        # Retrieve context
        context = self.retrieve_context(query, k)

        # Generate prompt based on query analysis
        if query_analysis['needs_structured'] and not query_analysis['needs_unstructured']:
            prompt = self.generate_structured_prompt(query, context['structured'])
            if not prompt:
                prompt = self.generate_hybrid_prompt(query, context)
        elif query_analysis['needs_unstructured'] and not query_analysis['needs_structured']:
            prompt = self.generate_unstructured_prompt(query, context['unstructured'])
            if not prompt:
                prompt = self.generate_hybrid_prompt(query, context)
        else:
            prompt = self.generate_hybrid_prompt(query, context)

        # Generate answer
        generated_text = self.generator(
            prompt,
            max_length=512,
            num_return_sequences=1,
            temperature=0.7,
            do_sample=True
        )[0]['generated_text']

        return {
            'query': query,
            'answer': generated_text,
            'context': context,
            'query_analysis': query_analysis,
            'prompt': prompt,
            'sources': {
                'structured_count': len(context['structured']),
                'unstructured_count': len(context['unstructured'])
            }
        }

# Example structured data
SAMPLE_STRUCTURED_DATA = [
    {"employee_id": 1, "name": "John Doe", "department": "Engineering", "salary": 75000},
    {"employee_id": 2, "name": "Jane Smith", "department": "Marketing", "salary": 65000},
    {"employee_id": 3, "name": "Bob Johnson", "department": "Engineering", "salary": 80000},
    {"employee_id": 4, "name": "Alice Brown", "department": "HR", "salary": 60000}
]

# Example unstructured data
SAMPLE_UNSTRUCTURED_DATA = """
Transformers are deep learning models that revolutionized natural language processing.
They use self-attention mechanisms to process sequential data.
The company policy states that all employees must complete annual training.
Engineering department focuses on product development and innovation.
Marketing department handles customer acquisition and brand management.
"""

def demo_hybrid_rag():
    """Demonstrate the hybrid RAG system"""
    # Initialize RAG system
    rag = AdvancedRAGSystem()

    print("=== Hybrid RAG System Demo ===")

    # Add structured data
    print("\n1. Adding structured data...")
    result = rag.add_structured_data(SAMPLE_STRUCTURED_DATA)
    print(result)

    # Add unstructured data
    print("\n2. Adding unstructured data...")
    result = rag.add_unstructured_data(SAMPLE_UNSTRUCTURED_DATA)
    print(result)

    # Test queries
    test_queries = [
        "How many employees are in the Engineering department?",
        "What is the average salary in the company?",
        "Explain what transformers are in AI?",
        "What does the Engineering department do?",
        "List all employees and their departments"
    ]

    for query in test_queries:
        print(f"\n{'='*50}")
        print(f"Query: {query}")

        result = rag.generate(query)

        print(f"Query Analysis: {result['query_analysis']}")
        print(f"Answer: {result['answer']}")
        print(f"Sources: {result['sources']}")

        # Show retrieved context
        print("\nRetrieved Context:")
        print("Structured:")
        for i, ctx in enumerate(result['context']['structured'][:2]):
            print(f"  {i+1}. {ctx['content'][:100]}... (score: {ctx['score']:.3f})")

        print("Unstructured:")
        for i, ctx in enumerate(result['context']['unstructured'][:2]):
            print(f"  {i+1}. {ctx['content'][:100]}... (score: {ctx['score']:.3f})")

# File-based data loading example
class FileDataLoader:
    """Helper class to load data from various file formats"""

    @staticmethod
    def load_text_file(file_path: str) -> str:
        with open(file_path, 'r', encoding='utf-8') as f:
            return f.read()

    @staticmethod
    def load_pdf(file_path: str) -> str:
        # Simple PDF text extraction (you might want to use PyPDF2 or pdfplumber)
        try:
            import PyPDF2
            with open(file_path, 'rb') as f:
                reader = PyPDF2.PdfReader(f)
                text = ""
                for page in reader.pages:
                    text += page.extract_text()
                return text
        except ImportError:
            print("PyPDF2 not installed. Install with: pip install PyPDF2")
            return ""

    @staticmethod
    def load_docx(file_path: str) -> str:
        try:
            import docx
            doc = docx.Document(file_path)
            return "\n".join([paragraph.text for paragraph in doc.paragraphs])
        except ImportError:
            print("python-docx not installed. Install with: pip install python-docx")
            return ""

if __name__ == "__main__":
    demo_hybrid_rag()

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

=== Hybrid RAG System Demo ===

1. Adding structured data...


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Added 4 structured documents

2. Adding unstructured data...


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Added 1 unstructured documents

Query: How many employees are in the Engineering department?
Query Analysis: {'needs_structured': True, 'needs_unstructured': False}
Answer: Alice Brown, department, HR; Alice Brown, salary, 60000
Sources: {'structured_count': 3, 'unstructured_count': 2}

Retrieved Context:
Structured:
  1. Structured data record: employee_id: 1, name: John Doe, department: Engineering, salary: 75000... (score: 0.426)
  2. Structured data record: employee_id: 3, name: Bob Johnson, department: Engineering, salary: 80000... (score: 0.405)
Unstructured:
  1. Transformers are deep learning models that revolutionized natural language processing. 
They use sel... (score: 0.437)
  2. Transformers are deep learning models that revolutionized natural language processing. 
They use sel... (score: -340282346638528859811704183484516925440.000)

Query: What is the average salary in the company?
Query Analysis: {'needs_structured': True, 'needs_unstructured': True}
Answer: The average